In [0]:
import warnings

warnings.filterwarnings('ignore')

%matplotlib inline 
import matplotlib.pyplot as plt 

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

In [0]:
pca_num=12

In [0]:
import requests
import io
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers
import matplotlib.pyplot as plt

def retrieve_data(urls):
  data = []
  for url in urls:
    response = requests.get(url)
    response.raise_for_status()
    data.append(np.load(io.BytesIO(response.content)))
  return data
  
images, labels = retrieve_data([
    "https://www.dropbox.com/s/zilveq4b7f1fdpz/images.npy?dl=1",
    "https://www.dropbox.com/s/6oue6rbyfcsatxg/labels.npy?dl=1"
])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, labels)

## min-max scaling

In [0]:
scaler = StandardScaler()
scaler.fit(X_train.reshape(len(X_train), -1, ))
X_train=scaler.transform(X_train.reshape(len(X_train), -1, ))

## PCA 

In [17]:
pca = PCA(n_components=pca_num)
pca.fit(X_train.reshape(len(X_train), -1, ))

PCA(copy=True, iterated_power='auto', n_components=12, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

> # Start **KFold**

In [0]:
folds = StratifiedKFold(n_splits=6, shuffle=True, random_state=1)

In [0]:
y = y_train

In [0]:
def do_train():
    oof_preds = np.zeros((len(X_train), ))
    preds = None

    for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
        train_objects = X_train[trn_]
        valid_objects = X_train[val_]
        y_train = y[trn_]
        y_valid = y[val_]
        print('train len', len(train_objects))
        print('valid len', len(valid_objects))
        
        trainX_pca = pca.transform(train_objects.reshape(len(train_objects), -1, ))
        validX_pca = pca.transform(valid_objects.reshape(len(valid_objects), -1, ))

        model = RandomForestClassifier(n_estimators=2000, max_depth=20)
        model.fit(trainX_pca, y_train)

        y_pred = model.predict_proba(validX_pca)[:, 1]
        print(y_valid.shape)
        print(y_pred.shape)
        current_loss = roc_auc_score(y_valid, y_pred)
        train_loss = roc_auc_score(y_valid, y_pred)
        print(f"k-f val AUC {current_loss}")
        print(f"train AUC {train_loss}")
        oof_preds[val_] = y_pred

        X_test_pca = pca.transform(X_test.reshape(len(X_test), -1, ))
        test_pred = model.predict_proba(X_test_pca)[:, 1]
        if preds is None:
            preds = test_pred
        else:
            preds += test_pred
        del model

    cv_loss = roc_auc_score(y, oof_preds)
    print('ALL FOLDS AUC: %.5f ' % cv_loss)
    oof_preds_df = pd.DataFrame()
    oof_preds_df['EventID'] = np.unique(data_train['EventID'].values)
    oof_preds_df['Proba'] = oof_preds
    oof_preds_df.to_csv('oof_preds.csv', index=False)
    return cv_loss, preds / folds.n_splits

In [30]:
best_loss, preds = do_train()
print('CV:', best_loss)

train len 50000
valid len 10000
(10000,)
(10000,)
k-f val AUC 0.8603313144132526
train AUC 0.8603313144132526
train len 50000
valid len 10000
(10000,)
(10000,)
k-f val AUC 0.864997474599899
train AUC 0.864997474599899
train len 50000
valid len 10000
(10000,)
(10000,)
k-f val AUC 0.8689007547560302
train AUC 0.8689007547560302
train len 50000
valid len 10000
(10000,)
(10000,)
k-f val AUC 0.8581617543264701
train AUC 0.8581617543264701
train len 50000
valid len 10000
(10000,)
(10000,)
k-f val AUC 0.85767468
train AUC 0.85767468
train len 50000
valid len 10000
(10000,)
(10000,)
k-f val AUC 0.8637692
train AUC 0.8637692
ALL FOLDS AUC: 0.86221 


NameError: ignored

In [0]:
submission = pd.DataFrame()
submission['EventID'] = np.unique(data_test['EventID'].values)
submission['Proba'] = preds
submission.to_csv(f's_pca{pca_num}.csv', index=False, float_format='%.6f')